CS512 - Computer Vision - Assignment 3 - S2

Submitted by-

Tushar Gwal A20449419

In [10]:
# Import necessary libraries
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Part 1: Data Loading and Preprocessing

Loading

In [11]:
original_dataset_dir = r'C:\Users\tusha\Downloads\kagglecatsanddogs_5340\PetImages'
base_dir = r'C:\Users\tusha\Downloads\kagglecatsanddogs_5340\subset_PetImages'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Creating directories
for dir in [train_dir, validation_dir, test_dir]:
    for animal in ['Cat', 'Dog']:
        os.makedirs(os.path.join(dir, animal), exist_ok=True)

def copy_pics(animal, num):
    src_dir = os.path.join(original_dataset_dir, animal)
    all_filenames = os.listdir(src_dir)
    random.shuffle(all_filenames)
    subset_filenames = all_filenames[:num]

    num_train = int(0.7 * num)
    num_val = int(0.15 * num)

    for i, filename in enumerate(subset_filenames):
        src_path = os.path.join(src_dir, filename)
        if i < num_train:
            dest_dir = train_dir
        elif i < num_train + num_val:
            dest_dir = validation_dir
        else:
            dest_dir = test_dir
        dest_path = os.path.join(dest_dir, animal, filename)
        shutil.copyfile(src_path, dest_path)

# Creating the subset dataset
for animal in ['Cat', 'Dog']:
    copy_pics(animal, num=2000)

Preprocessing

In [12]:
train_dir = r'C:\Users\tusha\Downloads\kagglecatsanddogs_5340\subset_PetImages\test'
validation_dir = r'C:\Users\tusha\Downloads\kagglecatsanddogs_5340\subset_PetImages\validation'
test_dir = r'C:\Users\tusha\Downloads\kagglecatsanddogs_5340\subset_PetImages\test'
img_size = (150, 150)
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)



Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


# Part 2: Build and train basic CNN model



In [13]:
from keras import models, layers
m1 =models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
m1.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
m1.summary()

#steps_per_epoch = len(train_generator)
#validation_steps = len(validation_generator)
history1 = m1.fit(
    train_generator,
    #steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator)
    #validation_steps=validation_steps)

val_loss, val_acc = m1.evaluate(validation_generator, steps=len(validation_generator))
print(f"Validation Accuracy: {val_acc * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

c:\Users\tusha\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.5011 - loss: 0.7126

c:\Users\tusha\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


19/19 ━━━━━━━━━━━━━━━━━━━━ 16s 798ms/step - accuracy: 0.5011 - loss: 0.7121 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 173ms/step - accuracy: 0.5207 - loss: 0.6943 - val_accuracy: 0.5000 - val_loss: 0.6920
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step - accuracy: 0.5031 - loss: 0.6905 - val_accuracy: 0.5550 - val_loss: 0.6908
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 177ms/step - accuracy: 0.5649 - loss: 0.6894 - val_accuracy: 0.5167 - val_loss: 0.6909
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step - accuracy: 0.5058 - loss: 0.6865 - val_accuracy: 0.5617 - val_loss: 0.6753
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.6297 - loss: 0.6487 - val_accuracy: 0.5950 - val_loss: 0.6539
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 186ms/step - accuracy: 0.6649 - loss: 0.6163 - val_accuracy: 0.6133 - val_loss: 0.6671
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 186ms/step - accuracy: 0.6914 - loss: 0.5960 - val_accuracy: 0.5733 - val

In [14]:
#evaluating the basic cnn model
t_loss, t_acc = m1.evaluate(test_generator, steps=len(test_generator))
print(f"Testing Accuracy for basic cnn model: {t_acc * 100:.2f}%")
print(f"Testing Loss for basic cnn model: {t_loss:.4f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.7967 - loss: 0.4636
Testing Accuracy for basic cnn model: 78.17%
Testing Loss for basic cnn model: 0.4729


# Part 3: Data Augmentation and Fine-tuning

In [15]:
train_dg_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator_augmented = train_dg_aug.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

m2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

m2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

steps_per_epoch = len(train_generator_augmented)
validation_steps = len(validation_generator)

history_augmented = m2.fit(
    train_generator_augmented,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

val_loss, val_acc = m2.evaluate(validation_generator, steps=len(validation_generator))
print(f"Validation Accuracy: {val_acc * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 316ms/step - accuracy: 0.4384 - loss: 0.7732 - val_accuracy: 0.5000 - val_loss: 0.6927
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 251ms/step - accuracy: 0.5208 - loss: 0.6900 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - accuracy: 0.4894 - loss: 0.6949 - val_accuracy: 0.5000 - val_loss: 0.6924
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 277ms/step - accuracy: 0.4987 - loss: 0.6920 - val_accuracy: 0.5583 - val_loss: 0.6871
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 48us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━

In [16]:
#evaluating the Augumented model
t_loss, t_acc = m2.evaluate(test_generator, steps=len(test_generator))
print(f"Testing Accuracy for augumented model: {t_acc * 100:.2f}%")
print(f"Testing Loss for augumented model: {t_loss:.4f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.5807 - loss: 0.6595
Testing Accuracy for augumented model: 56.17%
Testing Loss for augumented model: 0.6661


# Part 4: VGG16 Model

In [17]:
from tensorflow.keras import optimizers
base = VGG16(weights='imagenet',include_top=False, input_shape=(150, 150, 3))
base.trainable = False
m3 = models.Sequential([
    base,
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
m3.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])
m3.summary()
history_vgg16 = m3.fit(
    train_generator_augmented,
    steps_per_epoch=len(train_generator_augmented),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

val_loss, val_acc = m3.evaluate(validation_generator, steps=len(validation_generator))
print(f"Validation Accuracy: {val_acc * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,910,017 (72.14 MB)

 Trainable params: 4,195,329 (16.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.5413 - loss: 1.8756 - val_accuracy: 0.5867 - val_loss: 0.6075
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.6657 - loss: 0.5948 - val_accuracy: 0.8033 - val_loss: 0.4265
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.7544 - loss: 0.5024 - val_accuracy: 0.7917 - val_loss: 0.4095
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.7966 - loss: 0.4404 - val_accuracy: 0.8517 - val_loss: 0.3365
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.8424 - loss: 0.3672 - val_accuracy: 0.8483 - val_loss: 0.3218

In [18]:
#evaluating the VGG16 model
t_loss, t_acc = m3.evaluate(test_generator, steps=len(test_generator))
print(f"Testing Accuracy for VGG16 model: {t_acc * 100:.2f}%")
print(f"Testing Loss for VGG16 model: {t_loss:.4f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.8995 - loss: 0.2436
Testing Accuracy for VGG16 model: 90.50%
Testing Loss for VGG16 model: 0.2392


# Part 5: Evaluation and report results

1. Basic CNN Model: This basic CNN model has several layers that help it pick out important features from images. It includes convolutional, max-pooling, and dense layers to help it learn how to classify images.

Test Results
* Accuracy: 78.17%
* Loss: 0.4729

Analysis
* The basic CNN performs pretty well for a simple model, achieving an accuracy of 67.78%. However, the loss of 0.5964 suggests that it still struggles a bit to fully differentiate between the two classes. It does a decent job, but it may not be sophisticated enough to catch more complex patterns, which affects its accuracy.

2. Augmented CNN Model: This model adds data augmentation, which means the training images are randomly rotated, shifted, zoomed, and flipped to make the training set more diverse. The idea is to help the model perform better by learning from a wider variety of images.

Test Results
* Accuracy: 56.17%
* Loss: 0.6661

Analysis
*  This model didn’t do as well as the basic CNN. The accuracy dropped to 56.93%, and the loss increased to 0.6900, indicating that the model struggled with the augmented data. It could be overfitting, or maybe the augmentation techniques weren't applied in the best way, leading to worse performance.

3. VGG16-based CNN Model: This model swaps out the original CNN layers with a pre-trained VGG16 model, which has already learned useful patterns from a much larger dataset. By combining VGG16 with new layers for our specific task, the model can use transfer learning to improve performance.

Test Results
* Accuracy: 90.50%
* Loss: 0.2392

Analysis: 
* This model performs the best, with a high accuracy of 90.98% and a low loss of 0.2336. The big improvement is expected since it is using a pre-trained model which helps the network learn more complex features without starting from scratch.


Conclusion:
* Basic CNN Model: This model performs reasonably well for a straightforward architecture, achieving a test accuracy of 67.78%. However, it could benefit from more advanced feature extraction techniques to enhance its performance.

* Augmented CNN Model: The introduction of data augmentation resulted in lower performance, reducing test accuracy to 56.93%. This suggests that the augmentation methods may not have been optimally applied, or the model may have overfitted to the training data.

* VGG16-based CNN Model: This model delivered the best results, with a test accuracy of 90.98%. The application of transfer learning enabled the model to learn complex features more effectively, significantly improving accuracy and reducing test loss.